## IMPORTAR LAS LIBRERIAS

Actualizar a las que se usen finalmente en tu proyecto.

In [147]:
import numpy as np
import pandas as pd
import cloudpickle

#Automcompletar rápido
%config IPCompleter.greedy=True

from janitor import clean_names

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

## CARGAR LOS DATOS

### Ruta del proyecto

In [148]:
ruta_proyecto = r'C:\Users\mcent\OneDrive\Escritorio\PROYECTOS ML\SCORING_DE_RIESGOS'

### Nombre del fichero de datos

In [149]:
nombre_fichero_datos = 'prestamos.csv'

### Cargar los datos

In [150]:
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

df = pd.read_csv(ruta_completa,index_col=0)

### Seleccionar solo las variables finales

Se analizan los ficheros de df_pd, df_ead y df_lg para determinar las variables finales.

In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 137387967 to 94394801
Data columns (total 24 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   empleo                       185300 non-null  object 
 1   antigüedad_empleo            187112 non-null  object 
 2   ingresos                     200000 non-null  float64
 3   ingresos_verificados         200000 non-null  object 
 4   rating                       200000 non-null  object 
 5   dti                          199853 non-null  float64
 6   vivienda                     200000 non-null  object 
 7   num_hipotecas                195595 non-null  float64
 8   num_lineas_credito           199997 non-null  float64
 9   porc_tarjetas_75p            193368 non-null  float64
 10  porc_uso_revolving           199846 non-null  float64
 11  num_cancelaciones_12meses    199988 non-null  float64
 12  num_derogatorios             199997 non-null  fl

#### Apuntar (manualmente) la lista de variables finales sin extensiones

In [152]:
variables_finales = ['ingresos_verificados',
                     'vivienda',
                     'finalidad',
                     'num_cuotas',
                     'antiguedad_empleo',
                     'rating',
                     'ingresos',
                     'dti',
                     'num_lineas_credito',
                     'porc_uso_revolving',
                     'principal',
                     'tipo_interes',
                     'imp_cuota',
                     'num_derogatorios',
                     'estado',
                     'imp_amortizado',
                     'imp_recuperado',
                     'num_hipotecas']

**Se eliminan número de hipotecas, porc_tarjetas_75p**

#### Crear la matriz de variables procesos (excel)

Ir a la plantilla de Excel "Fase Producción Plantilla Procesos" y crear la matriz de variables por procesos.

#### Actualizar las importaciones

Ir arriba a la celda de importacion de paquetes y actualizarlos con los que finalmente vamos a usar.

## ESTRUCTURA DE LOS DATASETS

### Corregir los nombres

In [153]:
df = clean_names(df)

In [154]:
df = df[variables_finales]

### Eliminar registros

In [155]:
a_eliminar = ['porc_uso_revolving','dti','num_lineas_credito','num_derogatorios']

def eliminar_registros(temp, a_eliminar):
    for variable in a_eliminar:
        indices = temp.loc[df[variable].isna()].index
        temp.drop(indices, inplace= True)
        
eliminar_registros(df, a_eliminar)


a_dejar = df.loc[~((df.ingresos < 12000) | (df.ingresos > 300000))].index
df = df.loc[a_dejar]

## CREAR EL PIPELINE

### Instanciar calidad de datos

#### Crear la función

In [156]:
def calidad_datos(temp):
    
    temp = temp.astype({'num_hipotecas':'Int64','num_lineas_credito':'Int64',
                'num_derogatorios':'Int64'})
    
    temp['num_hipotecas'] = temp['num_hipotecas'].fillna(0)
    
    def imputar_moda(variable):
        return(variable.fillna(variable.mode()[0]))
    temp['antiguedad_empleo'] = imputar_moda(temp.antiguedad_empleo)
    
    
    temp['finalidad'] = temp.finalidad.replace({'house':'other','renewable_energy':'other'})
    temp['vivienda'] = temp.vivienda.replace({'NONE':'MORTGAGE','ANY':'MORTGAGE','OTHER':'MORTGAGE'})    
    
    minimo = 0
    maximo = 100           
    temp['dti'] = temp['dti'].clip(minimo,maximo)
    temp['porc_uso_revolving'] = temp['porc_uso_revolving'].clip(minimo,maximo)   
    
    temp['num_hipotecas'] = temp['num_hipotecas'].fillna(0)
    
    return(temp)

### Instanciar Creacion de Variables

Debido a que la creación de variables es diferentes para los 3 modelos, ya que hay tres targets, necesitamos construir 3 funciones.

#### Creacion funciones PD - EAD - LG

In [157]:
def creacion_pd(df):
    temp = df.copy()
    
    temp['pendiente'] = temp['principal'] - temp['imp_amortizado']
    temp['pd'] = np.where(temp.estado.isin(['Charged Off','Does not meet the credit policy. Status:Charged Off','Default']), 1, 0)

    #Eliminamos variables que ya no usaremos
    temp.drop(columns=['imp_recuperado','imp_amortizado','pendiente','estado'], inplace = True)
    
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return (temp_x, temp_y)

In [158]:
def creacion_ead(df):
    temp = df.copy()
    
    temp['ead'] = (1- temp.imp_amortizado/temp.principal)
    
    #Eliminamos variables que ya no usaremos
    temp.drop(columns=['imp_recuperado','imp_amortizado','estado'], inplace = True)
    
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return (temp_x, temp_y)

In [159]:
def creacion_lg(df):
    temp = df.copy()
    
    temp['pendiente'] = temp.principal - temp.imp_amortizado
    temp['lg'] = (1 - temp.imp_recuperado/temp.pendiente)
    temp['lg'].fillna(0,inplace=True)
    
    #Eliminamos variables que ya no usaremos
    temp.drop(columns=['imp_recuperado','imp_amortizado','pendiente','estado'], inplace = True)
    
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return (temp_x, temp_y)

#### Creamos dataframes x e y

In [160]:
x_pd, y_pd = creacion_pd(calidad_datos(df))

x_ead, y_ead = creacion_ead(calidad_datos(df))

x_lg, y_lg = creacion_lg(calidad_datos(df))

### Instanciar transformación de variables

In [135]:
## ONE HOT ENCODING
var_ohe = ['ingresos_verificados','vivienda','finalidad','num_cuotas']
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')


## ORDINAL ENCODING
var_oe = ['rating','antiguedad_empleo']
orden_rating = ['G','F','E','D','C','B','A']
orden_antiguedad_empleo = ['< 1 year', '1 year','2 years','3 years','4 years','5 years', '6 years', '7 years', '8 years', '9 years', '10+ years']

oe = OrdinalEncoder(categories = [orden_rating,orden_antiguedad_empleo],
                    handle_unknown = 'use_encoded_value',
                    unknown_value = 11)

#BINARIZACION
var_bin = ['num_derogatorios']
bin = Binarizer(threshold=0)

#ESTANDARIZACION
var_mms = ['ingresos','dti','num_hipotecas','num_lineas_credito','porc_uso_revolving','principal','tipo_interes','imp_cuota']
mms = MinMaxScaler()

### Crear el pipe del preprocesamiento

#### Crear el column transformer

In [136]:
ct = make_column_transformer(
    (ohe, var_ohe),
    (oe, var_oe),
    (bin, var_bin),
    (mms, var_mms),
    remainder='passthrough')

### Instanciar los modelos

#### Instanciar los algoritmos

In [137]:
modelo_pd = LogisticRegression(C=1, n_jobs=-1, penalty='l1', solver='saga')

modelo_ead = HistGradientBoostingRegressor(learning_rate=0.1,
                                           max_depth= 20, 
                                           max_iter=200, 
                                           l2_regularization=0.25, 
                                           min_samples_leaf=200, 
                                           scoring='neg_mean_absolute_percentage_error')

modelo_lg = HistGradientBoostingRegressor(learning_rate=0.1,
                                           max_depth= 20, 
                                           max_iter=200, 
                                           l2_regularization=0, 
                                           min_samples_leaf=200, 
                                           scoring='neg_mean_absolute_percentage_error')

#### Crear los pipes finales de entrenamiento

In [138]:
pipe_entrenamiento_pd = make_pipeline(ct,modelo_pd)
pipe_entrenamiento_ead = make_pipeline(ct,modelo_ead)
pipe_entrenamiento_lg = make_pipeline(ct,modelo_lg)

#### Guardar los pipes finales de entrenamiento

In [139]:
nombre_pipe_entrenamiento_pd = 'pipe_entrenamiento_pd.pickle'
nombre_pipe_entrenamiento_ead = 'pipe_entrenamiento_ead.pickle'
nombre_pipe_entrenamiento_lg = 'pipe_entrenamiento_lg.pickle'

ruta_pipe_entrenamiento_pd = ruta_proyecto + '/04_Modelos/' + nombre_pipe_entrenamiento_pd
ruta_pipe_entrenamiento_ead = ruta_proyecto + '/04_Modelos/' + nombre_pipe_entrenamiento_ead
ruta_pipe_entrenamiento_lg = ruta_proyecto + '/04_Modelos/' + nombre_pipe_entrenamiento_lg

with open(ruta_pipe_entrenamiento_pd, mode='wb') as file:
   cloudpickle.dump(pipe_entrenamiento_pd, file)
with open(ruta_pipe_entrenamiento_ead, mode='wb') as file:
   cloudpickle.dump(pipe_entrenamiento_ead, file)
with open(ruta_pipe_entrenamiento_lg, mode='wb') as file:
   cloudpickle.dump(pipe_entrenamiento_lg, file)

#### Entrenar los pipe final de ejecución

In [140]:
pipe_ejecucion_pd = pipe_entrenamiento_pd.fit(x_pd,y_pd)
pipe_ejecucion_ead = pipe_entrenamiento_ead.fit(x_ead,y_ead)
pipe_ejecucion_lg = pipe_entrenamiento_lg.fit(x_lg,y_lg)

C:\Users\mcent\miniconda3\envs\riesgos\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## GUARDAR EL PIPE

### Nombre del pipe final de ejecución

In [141]:
nombre_pipe_ejecucion_pd = 'pipe_ejecucion_pd.pickle'
nombre_pipe_ejecucion_ead = 'pipe_ejecucion_ead.pickle'
nombre_pipe_ejecucion_lg = 'pipe_ejecucion_lg.pickle'

### Guardar el pipe final de ejecución

In [142]:
ruta_pipe_ejecucion_pd = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion_pd

with open(ruta_pipe_ejecucion_pd, mode='wb') as file:
   cloudpickle.dump(pipe_ejecucion_pd, file)

ruta_pipe_ejecucion_ead = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion_ead

with open(ruta_pipe_ejecucion_ead, mode='wb') as file:
   cloudpickle.dump(pipe_ejecucion_ead, file)

ruta_pipe_ejecucion_lg = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion_lg

with open(ruta_pipe_ejecucion_lg, mode='wb') as file:
   cloudpickle.dump(pipe_ejecucion_lg, file)